In [14]:
import json
import os
import numpy as np
import pandas as pd
import plotnine
from plotnine import *  # Provides a ggplot-like interface to matplotlib.

# Get the BigQuery curated dataset for the current workspace context.
CDR = os.environ['WORKSPACE_CDR']

## Plot setup.
theme_set(theme_bw(base_size = 11)) # Default theme for plots.

def get_boxplot_fun_data(df):
  """Returns a data frame with a y position and a label, for use annotating ggplot boxplots.

  Args:
    d: A data frame.
  Returns:
    A data frame with column y as max and column label as length.
  """
  d = {'y': max(df), 'label': f'N = {len(df)}'}
  return(pd.DataFrame(data=d, index=[0]))

## ---------------[ CHANGE THESE AS NEEDED] ---------------------------------------
# Set default parameter values so that all snippets run successfully with no edits needed.
COHORT_QUERY = f'SELECT person_id FROM `{CDR}.person`'  # Default to all participants.
MEASUREMENT_OF_INTEREST = 'hemoglobin'
# Tip: the next four parameters could be set programmatically using one row from
# the result of measurements_of_interest_summary.sql
MEASUREMENT_CONCEPT_ID = 3004410        # Hemoglobin A1c
UNIT_CONCEPT_ID = 8554                  # percent
MEASUREMENT_NAME = '<this should be the measurement name>'
UNIT_NAME = '<this should be the unit name>'


In [15]:
from __future__ import print_function, division
import numpy as np
import scipy as sp

In [16]:
from scipy import stats

In [17]:

number_of_participants_with_measurements_df = pd.io.gbq.read_gbq(f'''

-- Compute the count of unique participants in our All of Us cohort
-- that have at least one measurement.
SELECT
  COUNT(DISTINCT person_id) AS number_of_participants_with_measurements
FROM
  `{CDR}.measurement`
WHERE
  person_id IN ({COHORT_QUERY})

''',
  dialect='standard')

number_of_participants_with_measurements_df.head()

,number_of_participants_with_measurements
0,341109


In [18]:

number_of_participants_with_med_conditions_df = pd.io.gbq.read_gbq(f'''

-- Compute the count of unique participants in our All of Us cohort
-- that have at least one condition.
SELECT
  COUNT(DISTINCT person_id) AS number_of_participants_with_med_conditions
FROM
  `{CDR}.condition_occurrence`
WHERE
  person_id IN ({COHORT_QUERY})

''',
  dialect='standard')

number_of_participants_with_med_conditions_df.head()

,number_of_participants_with_med_conditions
0,254487


In [20]:

measurements_of_interest_summary_df = pd.io.gbq.read_gbq(f'''

-- Compute summary information for our measurements of interest for our cohort.
--
-- PARAMETERS:
--   MEASUREMENT_OF_INTEREST: a case-insensitive string, such as "hemoglobin", to be compared
--                            to all measurement concept names to identify those of interest

WITH
  --
  -- Use a case insensitive string to search the measurement concept names of those
  -- measurements we do have in the measurements table.
  --
  labs_of_interest AS (
  SELECT
    measurement_concept_id,
    measurement_concept.concept_name AS measurement_name,
    unit_concept_id,
    unit_concept.concept_name AS unit_name
  FROM
    `{CDR}.measurement`
  LEFT JOIN `{CDR}.concept` AS measurement_concept
  ON measurement_concept.concept_id = measurement_concept_id
  LEFT JOIN `{CDR}.concept` AS unit_concept
  ON unit_concept.concept_id = unit_concept_id
  WHERE
    REGEXP_CONTAINS(measurement_concept.concept_name, r"(?i){MEASUREMENT_OF_INTEREST}")
  GROUP BY
    measurement_concept_id,
    unit_concept_id,
    measurement_concept.concept_name,
    unit_concept.concept_name
)
  --
  -- Summarize the information about each measurement concept of interest that our
  -- prior query identified.
  --
SELECT
  measurement_name AS measurement,
  IFNULL(unit_name, "NA") AS unit,
  COUNT(1) AS N,
  COUNTIF(value_as_number IS NULL
    AND (value_as_concept_id IS NULL
      OR value_as_concept_id = 0)) AS missing,
  AVG(value_as_number) AS avg,
  STDDEV(value_as_number) AS stddev,
  APPROX_QUANTILES(value_as_number, 4) AS quantiles,
  COUNTIF(value_as_number IS NOT NULL) AS num_numeric_values,
  COUNTIF(value_as_concept_id IS NOT NULL
      AND value_as_concept_id != 0) AS num_concept_values,
  COUNTIF(operator_concept_id IS NOT NULL) AS num_operators,
  IF(src_id = "PPI/PM", "PPI", "EHR") AS measurement_source,
  measurement_concept_id,
  unit_concept_id
FROM
  `{CDR}.measurement`
INNER JOIN
 labs_of_interest USING(measurement_concept_id, unit_concept_id)
LEFT JOIN
  `{CDR}.measurement_ext` USING(measurement_id)
WHERE
  person_id IN ({COHORT_QUERY})
GROUP BY
  measurement_concept_id,
  measurement_name,
  measurement_source,
  unit_concept_id,
  unit_name
ORDER BY
  N DESC


''',
  dialect='standard')

measurements_of_interest_summary_df.head()

,measurement,unit,N,missing,avg,stddev,quantiles,num_numeric_values,num_concept_values,num_operators,measurement_source,measurement_concept_id,unit_concept_id
0,Hemoglobin [Mass/volume] in Blood,gram per liter,3781591,6558,1111.277362,314703.065280,"[0.0, 105.0, 124.0, 137.0, 100000000.0]",3735894,260520,3546002,EHR,3000963,8636
1,Hemoglobin A1c/Hemoglobin.total in Blood,percent,528010,587,7965.617658,282002.651338,"[0.0, 5.6, 6.2, 7.3, 10000000.0]",512635,53186,495874,EHR,3004410,8554
2,Hemoglobin [Presence] in Urine by Test strip,No matching concept,356560,90098,26.837521,74.270368,"[0.0, 0.0, 0.0, 0.0, 500.0]",10859,260552,252267,EHR,3011397,0
3,Hemoglobin [Mass/volume] in Blood,No matching concept,301308,795,11.787695,2.659266,"[0.0, 10.2, 12.1, 13.6, 186.0]",298208,3501,301127,EHR,3000963,0
4,Glucose mean value [Mass/volume] in Blood Esti...,milligram per deciliter,203319,54,143.237250,45.827871,"[0.0, 114.0, 128.37, 157.07, 1001.0]",203178,30506,191825,EHR,3005131,8840
